In [ ]:
!pip install pyspark

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#folder_path = '/content/drive/My Drive/531_Final_Project

Mounted at /content/drive


In [ ]:
from pyspark import SparkContext, SparkConf
import sys
import csv
from itertools import islice, combinations
import time
import random
import itertools
import json
import xgboost as xgb
from xgboost import XGBRegressor
import numpy as np

if __name__ == '__main__':
  start_time = time.time()
  conf = SparkConf().setAppName('531_final_proj').setMaster('local[*]')
  sc = SparkContext.getOrCreate(conf=conf)
  sc.setSystemProperty('spark.driver.memory', '4g')
  sc.setSystemProperty('spark.executor.memory', '4g')
  sc.setLogLevel("ERROR")


lines = sc.textFile('/content/drive/My Drive/531_Final_Project/patients_data.csv')

column_names = lines.first()

rdd = lines.filter(lambda row: row != column_names).map(lambda row: row.split(','))

first18_rdd = lines.filter(lambda row: row != column_names).map(lambda row: row.split(',')[1:18])

In [ ]:
rdd.take(1)

[['0',
  'B',
  '4',
  '40',
  'Male',
  'Hispanic or Latino',
  'White or Caucasian',
  'English',
  'None',
  'Single',
  'Full Time',
  'Other',
  'Discharge',
  'Walk-in',
  'June',
  'Tuesday',
  '23-02',
  'No previous dispo',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  

In [ ]:
# dropping columns 2, 3, 6, 13 where NaN
pre_processed_rdd = rdd.filter(lambda row: all(row[index] != '' for index in [2, 3, 6, 12]))

In [ ]:
pre_processed_rdd.take(1)

[['0',
  'B',
  '4',
  '40',
  'Male',
  'Hispanic or Latino',
  'White or Caucasian',
  'English',
  'None',
  'Single',
  'Full Time',
  'Other',
  'Discharge',
  'Walk-in',
  'June',
  'Tuesday',
  '23-02',
  'No previous dispo',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  

In [ ]:
def convert_columns_rdd(row):
    return row[:1] + [float(row[i]) if i in [2, 3] else row[i] for i in range(1, len(row))]

rdd = pre_processed_rdd.map(convert_columns_rdd)

In [ ]:
# STRATIFIED SAMPLING
column_12_values = pre_processed_rdd.map(lambda row: row[12])
distribution = column_12_values.countByValue()

for value, count in distribution.items():
    print(f"Value: {value}, Count: {count}")

Value: Discharge, Count: 391979
Value: Admit, Count: 166013


In [ ]:
# keeping 20% of data
discharge_count = 391979
admit_count = 166013

total_count = discharge_count + admit_count


discharge_fraction_original = discharge_count / total_count
admit_fraction_original = admit_count / total_count

discharge_fraction_20 = discharge_fraction_original * 0.2
admit_fraction_20 = admit_fraction_original * 0.2

expected_sample_size_discharge = discharge_count * 0.2
expected_sample_size_admit = admit_count * 0.2

total_expected_sample_size = expected_sample_size_discharge + expected_sample_size_admit

(discharge_fraction_20, admit_fraction_20, expected_sample_size_discharge, expected_sample_size_admit, total_expected_sample_size)


(0.14049627951655222, 0.05950372048344779, 78395.8, 33202.6, 111598.4)

In [ ]:
pair_rdd = rdd.map(lambda row: (row[12], row))

fractions = {
    "Discharge": 0.14049627951655222,
    "Admit": 0.05950372048344779
}

stratified_sample_rdd = pair_rdd.sampleByKey(False, fractions)

sample_count = stratified_sample_rdd.count()

for sample in stratified_sample_rdd.take(5):
    print(sample)

Stratified sample count: 65151
('Discharge', ['0', 'B', 4.0, 40.0, 'Male', 'Hispanic or Latino', 'White or Caucasian', 'English', 'None', 'Single', 'Full Time', 'Other', 'Discharge', 'Walk-in', 'June', 'Tuesday', '23-02', 'No previous dispo', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '

# DATA PRE_PROCESSING
## DROP column 2 -- ESI, and make all NaN values in columns > 18 = 0 because they are medical and we assume not present binary

In [ ]:
og_stratified_rdd = stratified_sample_rdd.map(lambda kv: kv[1])

In [ ]:
og_stratified_rdd.count()

65151

In [ ]:
adjusted_rdd = og_stratified_rdd.map(lambda row: row[:18] + [0.0 if item == '' else item for item in row[18:]])

adjusted_rdd.take(1)

[['0',
  'B',
  4.0,
  40.0,
  'Male',
  'Hispanic or Latino',
  'White or Caucasian',
  'English',
  'None',
  'Single',
  'Full Time',
  'Other',
  'Discharge',
  'Walk-in',
  'June',
  'Tuesday',
  '23-02',
  'No previous dispo',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  

In [ ]:
def convert_columns_rdd(row):
    return row[:1] + [float(row[i]) if i in [2, 3] else row[i] for i in range(1, len(row))]

adjusted_rdd = adjusted_rdd.map(convert_columns_rdd)

In [ ]:
# TRAIN TEST SPLIT
preprocessed_rdd_no_first_column = adjusted_rdd.map(lambda row: row[1:])

train_rdd, test_rdd = preprocessed_rdd_no_first_column.randomSplit(weights=[0.7, 0.3], seed=42) # 70 / 30 split

In [ ]:
test_rdd.take(1)

[['A',
  2.0,
  27.0,
  'Female',
  'Non-Hispanic',
  'White or Caucasian',
  'English',
  'Christian',
  'Single',
  'Not Employed',
  'Medicaid',
  'Discharge',
  'ambulance',
  'July',
  'Tuesday',
  '15-18',
  'Discharge',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '1.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',

In [ ]:
# Label
X_train = train_rdd.map(lambda row: row[:10] + row[12:])
X_test = test_rdd.map(lambda row: row[:10] + row[12:])

y_train = train_rdd.map(lambda row: row[11]) # index 11 is discharge
y_test = test_rdd.map(lambda row: row[11])

In [ ]:
y_test.take(1)

['Discharge']

# ONE HOT ENCODING

In [ ]:
# ONE HOT ENCODE X_train, X_test columns
# [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

columns_to_encode = [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] # check number


distinct_values = {}
for col_index in columns_to_encode:
    distinct_values[col_index] = X_train.map(lambda row: row[col_index]).distinct().collect()

distinct_values_bc = sc.broadcast(distinct_values)

def one_hot_encode_row(row):
    encoded_row = []
    for col_index, value in enumerate(row):
        if col_index in distinct_values_bc.value:
            categories = distinct_values_bc.value[col_index]
            encoded_vector = [1 if category == value else 0 for category in categories]
            encoded_row.extend(encoded_vector)
        else:
            encoded_row.append(value)
    return encoded_row


X_train_encoded = X_train.map(one_hot_encode_row)

In [ ]:
X_train_encoded.take(1)

[[0,
  0,
  1,
  4.0,
  40.0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.

In [ ]:
columns_to_encode = [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

distinct_values = {}
for col_index in columns_to_encode:
    distinct_values[col_index] = X_test.map(lambda row: row[col_index]).distinct().collect()

distinct_values_bc = sc.broadcast(distinct_values)

def one_hot_encode_row(row):
    encoded_row = []
    for col_index, value in enumerate(row):
        if col_index in distinct_values_bc.value:
            categories = distinct_values_bc.value[col_index]
            encoded_vector = [1 if category == value else 0 for category in categories]
            encoded_row.extend(encoded_vector)
        else:
            encoded_row.append(value)
    return encoded_row

X_test_encoded = X_test.map(one_hot_encode_row)

In [ ]:
X_test_encoded.take(1)

[[1,
  0,
  0,
  2.0,
  27.0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '1.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.

In [ ]:
# one hot encode y_train and y_test
y_train_encoded = y_train.map(lambda status: 1 if status == 'Admit' else 0)
y_test_encoded = y_test.map(lambda status: 1 if status == 'Admit' else 0)

In [ ]:
X_test_encoded.take(1)

[[1,
  0,
  0,
  2.0,
  27.0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '1.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.

In [ ]:
def convert_row_to_floats(row):
    new_row = []
    for item in row:
        try:
            new_row.append(float(item))
        except ValueError:
            new_row.append(item)
    return new_row

X_train_encoded_floats = X_train_encoded.map(convert_row_to_floats)
X_test_encoded_floats =  X_test_encoded.map(convert_row_to_floats)


In [ ]:
X_train = X_train_encoded_floats.collect()
X_test = X_test_encoded_floats.collect()

y_train = y_train_encoded.collect()
y_test = y_test_encoded.collect()

params = {
    'objective': 'binary:hinge',
    'eval_metric': 'rmse',
}
xgb_model = xgb.XGBRegressor(**params)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, objective='binary:hinge', ...)

In [ ]:
preds = xgb_model.predict(X_test)

In [ ]:
print(preds)

[0. 0. 1. ... 0. 0. 0.]


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
print("RMSE : ", np.sqrt(mean_squared_error(y_test, preds)))

RMSE :  0.31512160532180644


In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, preds)
print("ACCURACY:", accuracy)

ACCURACY: 0.9006983738594077


In [ ]:
# Stat Parity
gender_and_preds = X_test_encoded_floats.map(lambda x: (x[5], x[6]))

gender_positive_rates = gender_and_preds.groupByKey().mapValues(lambda preds: sum(preds) / len(preds))

gender_positive_rates_collected = gender_positive_rates.collect()

statistical_parity_difference = abs(gender_positive_rates_collected[0][1] - gender_positive_rates_collected[1][1])

print("Statistical Parity Difference (Gender):", statistical_parity_difference)


Statistical Parity Difference (Gender): 1.0


In [ ]:
!pip install fairlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 41.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.26.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


In [ ]:
X_test

In [ ]:
# Equal Opprotunity Gender
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio

sens_data = X_test_encoded_floats.map(lambda x: (x[5], x[6]))  # Assuming gender is at index 4 and 5 in each data point

sens_data_collected = sens_data.collect()
scalar_sens_data = [gender[0] for gender in sens_data_collected]

print(equalized_odds_difference(y_test,
                                preds,
                                sensitive_features=scalar_sens_data))

print(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data))

0.008703470303512928
0.9435118068050816


In [ ]:
# Stat parity gender
from fairlearn.metrics import demographic_parity_ratio

sens_data = X_test_encoded_floats.map(lambda x: (x[5], x[6]))  # Assuming gender is at index 4 and 5 in each data point

sens_data_collected = sens_data.collect()
scalar_sens_data = [gender[0] for gender in sens_data_collected]

print(demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data))

0.9617352124422541


In [ ]:
# Stat Parity -> medicare vs medicaid (insurance status)

sens_data = X_test_encoded_floats.map(lambda x: (x[53:58]))  # Assuming gender is at index 4 and 5 in each data point

sens_data_collected = sens_data.collect()

scalar_sens_data = [status[0] for status in sens_data_collected]

print("equalized odds insurance status",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("stat parity insurance staus:", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))


equalized odds insurance status 0.5519104750332751
stat parity insurance staus: 0.5911032025771357


In [ ]:
# RACE: Stat Parity & equzlied odds -> race (11-17)
sens_data = X_test_encoded_floats.map(lambda x: (x[11:18]))  # Assuming gender is at index 4 and 5 in each data point

# Collect the results
sens_data_collected = sens_data.collect()
scalar_sens_data = [race[0] for race in sens_data_collected]


print("equalized odds race:",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("stat parity race:", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))



equalized odds race: 0.4285298822160826
stat parity race: 0.48116200674201864


In [ ]:
from fairlearn.metrics import demographic_parity_difference
# AGE
sens_data = X_test_encoded_floats.map(lambda x: x[4])
scalar_sens_data = sens_data.collect()


print("equalized odds age:",(equalized_odds_ratio(y_test,
                           preds,
                           sensitive_features=scalar_sens_data)))

print("stat parity ratio age:", (demographic_parity_ratio(y_test, preds, sensitive_features=scalar_sens_data)))


print("stat parity difference age:", (demographic_parity_difference(y_test, preds, sensitive_features=scalar_sens_data)))


equalized odds age: 0.0
stat parity ratio age: 0.0
stat parity difference age: 1.0
